In [1]:
# HIDE CODE
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
matplotlib.rcParams.update({'font.size': 12})


import warnings
import sys
import os
sys.path.append('/home/server/gli-data-science/')
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

import os
import ds_db
import helper_db
from helper import transform_to_rupiah, rupiah_format

import pickle
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display, HTML, display_html, IFrame
import ipywidgets as ipyw

def side_by_side_display(dfs:list, captions:list):
    output = ""
    combined = dict(zip(captions, dfs))
    styles = [
        dict(selector="caption", props=[("caption-side", "center"), ("font-size", "100%"), ("color", )])]
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline; font-size:85%' ").set_precision(2).set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0\xa0"

    display(HTML(output))

    
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, PoissonRegressor, Ridge, Lasso, RANSACRegressor, RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor, export_text, DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVR, SVC, LinearSVC
import xgboost as xgb 


from joblib import dump, load
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split


import textwrap
def split_label(list_label):
    list_label = list(list_label)
    list_label = ["<br>".join(textwrap.wrap(t, width=12)) for t in list_label ]
    return list_label

### ide kasar
> 
* daily job to loop all member alfagift 10 juta
* compute feature trx voucher used, # of trx, # of sales
* compute feature event app site open / view product / atc
* updating each feature and re predict churn feature
* lets play in month - 3 for each define trx and event feature

In [2]:
df_pv_ = pd.read_csv('./feature/pv_31Dec21.csv')
df_pv_['EVENT_TIME'] = pd.to_datetime(df_pv_['EVENT_TIME']).dt.strftime('%Y-%m')

df_pv = df_pv_.groupby(['TRO_MEMBERS','EVENT_TIME']).agg(PV=('COUNT_VIEW_PRODUCT','sum'))\
        .unstack(level=1).fillna(0).reset_index()

df_pv.columns = [' '.join(col).strip().upper() for col in df_pv.columns.values]

In [4]:
df_pv.head()

,TRO_MEMBERS,PV 2021-09,PV 2021-10,PV 2021-11,PV 2021-12
0,9990010000001200,25.0,27.0,46.0,38.0
1,9990010000004582,0.0,6.0,0.0,0.0
2,9990010000008534,0.0,4.0,0.0,0.0
3,9990010000010072,24.0,96.0,55.0,28.0
4,9990010000031123,0.0,0.0,1.0,1.0


In [3]:
df_re_g_u = pd.read_csv('./feature/rating_2022-01-01.csv')

In [5]:
df_re_g_u.head()

,TRO_MEMBERS,RATING_BELOW 2021-09,RATING_BELOW 2021-10,RATING_BELOW 2021-11,RATING_BELOW 2021-12,RATING_AVG 2021-09,RATING_AVG 2021-10,RATING_AVG 2021-11,RATING_AVG 2021-12
0,9990010000069277,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
1,9990010000070044,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
2,9990010000089999,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
3,9990010000191790,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0
4,9990010000196641,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0


In [6]:
# HIDE CODE

## USING ORDER
import glob

df_tto = []
for end_date in pd.date_range('2021-09-01', '2022-01-01', freq='M'):
    start_date = end_date.replace(day=1)
    end_date_save = start_date + relativedelta(months=1)
    
    start_date_str = start_date.strftime('%d%b%y')
    end_date_str = end_date_save.strftime('%d%b%y')
    
    print(start_date_str, end_date_str)
    
    pf = '/home/server/gli-data-science/data/trans_oshop_test/{}_{}.csv'.format(start_date_str, end_date_str)
    df_tto.append(pd.read_csv(pf, sep='\t', dtype='object'))
    
df_tto = pd.concat(df_tto)

# df_tto = df_tto[df_tto['TRO_MEMBERS'] != 'N']
df_tto['TRO_DATE'] = pd.to_datetime(df_tto['TRO_DATE'])
df_tto['TRO_NET'] = df_tto['TRO_NET'].astype('float', errors='ignore').astype('int', errors='ignore')


df_tto['TRO_QTY'] = df_tto['TRO_QTY'].astype(int)

01Sep21 01Oct21
01Oct21 01Nov21
01Nov21 01Dec21
01Dec21 01Jan22


In [7]:
## USE THIS FOR GET PV DATA

df_mem_push = df_tto[['TRO_MEMBERS']].drop_duplicates()
from sqlalchemy import event,create_engine,types



driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )

engine = create_engine(engine_stmt)

df = df_mem_push.astype(str)
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('temp_churn', engine, index=False, if_exists="replace", dtype=dtyp)

engine.dispose()





In [8]:
df_tto['TRO_VOUCHER_USAGE'] = df_tto['TRO_VOUCHER_USAGE']\
                            .astype('float', errors='ignore').astype('int', errors='ignore')
df_tto['TRO_DH_SPESIAL'] = df_tto['TRO_DH_SPESIAL']\
                            .astype('float', errors='ignore').astype('int', errors='ignore')


df_tto_sel = df_tto[(df_tto['TRO_DATE'] >= '2021-09-01') \
            & (df_tto['TRO_DATE'] <= '2021-12-31')]

In [9]:
df_sla = pd.read_csv('./feature/sla_31Dec21.csv')
df_sla['ATT_SEND_DATE_TOSTORE'] = pd.to_datetime(df_sla['ATT_SEND_DATE_TOSTORE'])
df_sla['SLA'] = np.where(df_sla['SLA_NORM']< 0, 20, df_sla['SLA_NORM'])

df_tto_sel = pd.merge(df_tto_sel, df_sla, left_on='TRO_NO_ORDER', right_on='ATT_ORDER_ID', how='left')

col_drop = [
    'ATT_ORDER_ID',
    'ATT_ORDER_DATE',
    'ATT_DELIVERY_DATE',
    'ATT_SEND_DATE_TOSTORE',
    'SLA_NORM'        
]
df_tto_sel = df_tto_sel.drop(col_drop,1)

## create feature
df_tto_sel['SLA'] = df_tto_sel['SLA'].fillna(20)
df_tto_sel['SLA_LATE'] = np.where(df_tto_sel['SLA'] > 60, 1,0)

In [10]:
q = '''
SELECT 
	tc.TRO_MEMBERS, 
	(TO_DATE('2021/12/08', 'yyyy/mm/dd')) - zt.tgl_registrasi AS SINCE_REGIS
FROM 
	TEMP_CHURN tc
LEFT JOIN (
	select 
		char1 no_member, char4 jenis_kelamin, char5 status_nikah, date1 tgl_lahir, date2 tgl_registrasi
	from 
		report.ztampungan
	where 
		pk='MEMBERALFAGIFT2'
) zt ON zt.no_member = tc.TRO_MEMBERS 

'''

con = ds_db.connect_alfabi()

df_regis = pd.read_sql_query(q, con)

con.close()

You are connected


In [11]:
df_regis['SINCE_REGIS'] = df_regis['SINCE_REGIS'].fillna('-1')

In [12]:
df_tto_sel_g = df_tto_sel.groupby(['TRO_MEMBERS','TRO_DATE_MONTH','TRO_NO_ORDER'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_DH_SPESIAL':'sum',
                              'TRO_NO_ORDER':'nunique',
                              'SLA':'first',
                              'SLA_LATE':'first',
                              'TRO_VOUCHER_USAGE':'sum',
                              'TRO_PAYMENT_METHOD':'first'
                              ''
                             })
df_tto_sel_g = df_tto_sel_g.groupby(['TRO_MEMBERS','TRO_DATE_MONTH'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_DH_SPESIAL':'sum',
                              'TRO_NO_ORDER':'sum',
                              'SLA':'sum',
                              'SLA_LATE':'sum',
                              'TRO_VOUCHER_USAGE':'sum',
                              'TRO_PAYMENT_METHOD':'nunique'
                             })

## unstack all feature in oshop

In [13]:
df_tto_sel_g_u = df_tto_sel_g.unstack(level=1).fillna(0).reset_index()
df_tto_sel_g_u.columns = [' '.join(col).strip().upper() for col in df_tto_sel_g_u.columns.values]

## merge with rating

In [14]:
df_re_g_u['TRO_MEMBERS'] = df_re_g_u['TRO_MEMBERS'].astype(str)
df_tto_re = pd.merge(df_tto_sel_g_u, df_re_g_u, on='TRO_MEMBERS', how='left').fillna(0)

## merge with product view

In [15]:
df_pv['TRO_MEMBERS'] = df_pv['TRO_MEMBERS'].astype(str)
df_tto_re_pv = pd.merge(df_tto_re, df_pv, on='TRO_MEMBERS', how='left').fillna(0)

## merge with regis

In [16]:
df_tto_re_pv = pd.merge(df_tto_re_pv, df_regis, on='TRO_MEMBERS')
df_tto_re_pv = df_tto_re_pv[df_tto_re_pv['SINCE_REGIS'].astype(int) > 0]

In [17]:
df_tto_re_pv.head()

,TRO_MEMBERS,TRO_NET 2021-09,TRO_NET 2021-10,TRO_NET 2021-11,TRO_NET 2021-12,TRO_DH_SPESIAL 2021-09,TRO_DH_SPESIAL 2021-10,TRO_DH_SPESIAL 2021-11,TRO_DH_SPESIAL 2021-12,TRO_NO_ORDER 2021-09,TRO_NO_ORDER 2021-10,TRO_NO_ORDER 2021-11,TRO_NO_ORDER 2021-12,SLA 2021-09,SLA 2021-10,SLA 2021-11,SLA 2021-12,SLA_LATE 2021-09,SLA_LATE 2021-10,SLA_LATE 2021-11,SLA_LATE 2021-12,TRO_VOUCHER_USAGE 2021-09,TRO_VOUCHER_USAGE 2021-10,TRO_VOUCHER_USAGE 2021-11,TRO_VOUCHER_USAGE 2021-12,TRO_PAYMENT_METHOD 2021-09,TRO_PAYMENT_METHOD 2021-10,TRO_PAYMENT_METHOD 2021-11,TRO_PAYMENT_METHOD 2021-12,RATING_BELOW 2021-09,RATING_BELOW 2021-10,RATING_BELOW 2021-11,RATING_BELOW 2021-12,RATING_AVG 2021-09,RATING_AVG 2021-10,RATING_AVG 2021-11,RATING_AVG 2021-12,PV 2021-09,PV 2021-10,PV 2021-11,PV 2021-12,SINCE_REGIS
1,9990010000001200,1791590.0,2412654.0,3117811.0,3217556.0,81500.0,109200.0,131400.0,138200.0,20.0,35.0,49.0,40.0,217.1,480.8,630.1,710.1,0.0,1.0,0.0,1.0,0.0,0.0,200000.0,85000.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,27.0,46.0,38.0,932.0
2,9990010000004582,0.0,191267.0,0.0,0.0,0.0,70400.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,9951.7,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,744.0
3,9990010000008534,202358.0,644400.0,165315.0,776032.0,5200.0,14300.0,2200.0,185500.0,4.0,10.0,3.0,8.0,73.3,214.9,93.3,159.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,70000.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,927.0
4,9990010000031123,0.0,0.0,412904.0,48908.0,0.0,0.0,124100.0,27000.0,0.0,0.0,4.0,1.0,0.0,0.0,34.6,359.2,0.0,0.0,0.0,1.0,0.0,0.0,10000.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,761.0
5,9990010000034518,48952.0,39453.0,65634.0,130813.0,12900.0,6200.0,400.0,7200.0,2.0,1.0,2.0,3.0,1868.2,437.0,11.3,103.2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,4.0,4.0,910.0


In [18]:
# df_tto_re_pv['NET 11-10'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-10']
# df_tto_re_pv['NET 10-09'] = df_tto_re_pv['TRO_NET 2021-10'] - df_tto_re_pv['TRO_NET 2021-09']
# df_tto_re_pv['NET 11-09'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-09']

# df_tto_re_pv['STRUK 11-10'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']
# df_tto_re_pv['STRUK 10-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-10'] - df_tto_re_pv['TRO_NO_ORDER 2021-09']
# df_tto_re_pv['STRUK 11-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-09']

# df_tto_re_pv['PV 11-10'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-10']
# df_tto_re_pv['PV 10-09'] = df_tto_re_pv['PV 2021-10'] - df_tto_re_pv['PV 2021-09']
# df_tto_re_pv['PV 11-09'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-09']

# df_tto_re_pv['SLA 11-10'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-10']
# df_tto_re_pv['SLA 10-09'] = df_tto_re_pv['SLA 2021-10'] - df_tto_re_pv['SLA 2021-09']
# df_tto_re_pv['SLA 11-09'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-09']

# df_tto_re_pv['BASKET 11'] = df_tto_re_pv['TRO_NET 2021-11']/df_tto_re_pv['TRO_NO_ORDER 2021-11']
# df_tto_re_pv['BASKET 10'] = df_tto_re_pv['TRO_NET 2021-10']/df_tto_re_pv['TRO_NO_ORDER 2021-10']
# df_tto_re_pv['BASKET 09'] = df_tto_re_pv['TRO_NET 2021-09']/df_tto_re_pv['TRO_NO_ORDER 2021-09']

# df_tto_re_pv['BASKET AVG'] = (df_tto_re_pv['BASKET 11'] + df_tto_re_pv['BASKET 10'] + df_tto_re_pv['BASKET 09']) / 3

In [19]:
# print(df_ins[df_ins['label'] == 1]['NET_11-10'].median())
# print(df_ins[df_ins['label'] == 1]['NET_10-09'].median())
# print(df_ins[df_ins['label'] == 1]['NET_11-09'].median())
# print(df_ins[df_ins['label'] == 1]['STRUK_11-10'].mean())
# print(df_ins[df_ins['label'] == 1]['STRUK_10-09'].mean())
# print(df_ins[df_ins['label'] == 1]['STRUK_11-09'].mean())

## Random Forest

### not oversampling version without threshold 2/3 or 3/3

In [21]:
from sklearn.model_selection import StratifiedKFold
condition_2_3 = (
((df_tto_re_pv['TRO_NET 2021-09'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0)) | \
    
((df_tto_re_pv['TRO_NET 2021-09'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] == 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0)) |
    
((df_tto_re_pv['TRO_NET 2021-09'] == 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0))
)

condition_3_3 = ( 
          (df_tto_re_pv['TRO_NET 2021-09'] > 0) & \
          (df_tto_re_pv['TRO_NET 2021-10'] > 0) & \
          (df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
          (df_tto_re_pv['TRO_NET 2021-12'] == 0) 
)



df_tto_re_pv['label'] = np.where(condition_2_3, 1, 0)


X = df_tto_re_pv.drop(
[
'TRO_MEMBERS',
'TRO_NET 2021-12', 
'TRO_DH_SPESIAL 2021-12',
'TRO_NO_ORDER 2021-12',
'SLA 2021-12',
'SLA_LATE 2021-12',
'RATING_BELOW 2021-12',
'RATING_AVG 2021-12',
'PV 2021-12',
'TRO_VOUCHER_USAGE 2021-12',
'TRO_PAYMENT_METHOD 2021-12',
'label'
], 1
)
y = df_tto_re_pv[['label']]


# X['NET-1'] = X.iloc[:,2] - X.iloc[:,1]
# X['NET-2'] = X.iloc[:,1] - X.iloc[:,0]


# X['ORDER-1'] = X.iloc[:,8] - X.iloc[:,7]
# X['ORDER-2'] = X.iloc[:,7] - X.iloc[:,6]


# X['SLA-1'] = X.iloc[:,11] - X.iloc[:,10]
# X['SLA-2'] = X.iloc[:,10] - X.iloc[:,9]


# X['PV-1'] = X.iloc[:,29] - X.iloc[:,28]
# X['PV-2'] = X.iloc[:,28] - X.iloc[:,27]


# skf = StratifiedKFold(n_splits=5)
# for train_index, test_index in skf.split(X, y):
#     print("="*50)
#     X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
#     y_train, y_test = y.iloc[train_index,:], y.iloc[test_index,:]


    
#     ## applying standard scaler
#     scaler = StandardScaler()
#     scaler.fit(X_train)
#     X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
#     X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_train.columns)
    

#     #clf_ridge = RandomForestClassifier(random_state=42) 
#     clf_ridge = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(70,))
#     clf_ridge.fit(X_train, y_train) 
#     pred = clf_ridge.predict(X_test)
#     print(classification_report(y_test, pred))
#     fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
#     ConfusionMatrixDisplay.from_predictions(y_test, pred, 
#                                             cmap='Blues', display_labels= ['not_churn', 'churn'], 
#                                             ax = ax, values_format = 'd')




# applying standard scaler
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)
dump(scaler, './model/scaler_70_dec21_2_3.joblib')


clf_all_mlp = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(70,))
clf_all_mlp.fit(X, y) 
dump(clf_all_mlp, './model/mlp_70_dec21_2_3.joblib')


['./model/mlp_70_dec21_2_3.joblib']

In [ ]:
# a = pd.DataFrame()

# a['NET 11-10'] = df_tto_re_pv['TRO_NET 2021-12'] - df_tto_re_pv['TRO_NET 2021-11']
# a['NET 10-09'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-10']
# a['NET 11-09'] = df_tto_re_pv['TRO_NET 2021-12'] - df_tto_re_pv['TRO_NET 2021-10']

# a['STRUK 11-10'] = df_tto_re_pv['TRO_NO_ORDER 2021-12'] - df_tto_re_pv['TRO_NO_ORDER 2021-11']
# a['STRUK 10-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']
# a['STRUK 11-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-12'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']

# a['PV 11-10'] = df_tto_re_pv['PV 2021-12'] - df_tto_re_pv['PV 2021-11']
# a['PV 10-09'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-10']
# a['PV 11-09'] = df_tto_re_pv['PV 2021-12'] - df_tto_re_pv['PV 2021-10']

# a['SLA 11-10'] = df_tto_re_pv['SLA 2021-12'] - df_tto_re_pv['SLA 2021-11']
# a['SLA 10-09'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-10']
# a['SLA 11-09'] = df_tto_re_pv['SLA 2021-12'] - df_tto_re_pv['SLA 2021-10']

# a['BASKET 11'] = df_tto_re_pv['TRO_NET 2021-12']/df_tto_re_pv['TRO_NO_ORDER 2021-12']
# a['BASKET 10'] = df_tto_re_pv['TRO_NET 2021-11']/df_tto_re_pv['TRO_NO_ORDER 2021-11']
# a['BASKET 09'] = df_tto_re_pv['TRO_NET 2021-10']/df_tto_re_pv['TRO_NO_ORDER 2021-10']

# a['BASKET AVG'] = (df_tto_re_pv['BASKET 11'] + df_tto_re_pv['BASKET 10'] + df_tto_re_pv['BASKET 09']) / 3

In [22]:
X_12 = df_tto_re_pv.drop(
[
'TRO_MEMBERS',
'TRO_NET 2021-09', 
'TRO_DH_SPESIAL 2021-09',
'TRO_NO_ORDER 2021-09',
'SLA 2021-09',
'SLA_LATE 2021-09',
'RATING_BELOW 2021-09',
'RATING_AVG 2021-09',
'PV 2021-09',
'TRO_VOUCHER_USAGE 2021-09',
'TRO_PAYMENT_METHOD 2021-09',
'label'
], 1
)
X_12['SINCE_REGIS'] = X_12['SINCE_REGIS']+30

# X_12 = pd.concat([X_12.iloc[:,0:-16],a],1)
# X_12 = X_12.fillna(0)
# y_12 = df_tto_re_pv['label']

In [ ]:
# X_12.head()

In [25]:
X_12.shape

(524348, 31)

In [26]:


scaler_all = load('./model/scaler_70_dec21_2_3.joblib')
clf_all = load('./model/mlp_70_dec21_2_3.joblib')


X_12_scale = pd.DataFrame(scaler_all.transform(X_12), index=X_12.index, columns=X_12.columns)
pred = (clf_all.predict_proba(X_12_scale)[:,1] >= 0.5).astype(int)


df_mem_12 = pd.concat([df_tto_re_pv[['TRO_MEMBERS']].reset_index(drop=True)
                       , pd.DataFrame(pred, columns=['PRED'])], 1)

In [27]:
df_mem_12[df_mem_12['PRED'] == 1].shape

(14432, 2)

In [28]:
df_trader = pd.read_excel('/home/server/gli-data-science/ferina/Sales Trader Ytd Aug-ExEmployee.xlsx'
                          , dtype='object', names=['member','stat'])

In [29]:
df_mem_12 = df_mem_12[~df_mem_12['TRO_MEMBERS'].isin(df_trader['member'])]

In [30]:
df_mem_12[df_mem_12['PRED'] == 1].shape

(14331, 2)

In [31]:
# pf = '/home/server/gli-data-science/data/trans_oshop_test/01Mar22_01Apr22.csv'
# df_feb = pd.read_csv(pf, sep='\t', dtype='object')

# df_feb_mem = df_feb['TRO_MEMBERS'].drop_duplicates()

# df_mem_12 = df_mem_12[~df_mem_12['TRO_MEMBERS'].isin(df_feb_mem)]

# df_churn_sel = df_mem_12[df_mem_12['PRED'] == 1].copy()

In [33]:
df_churn_sel['PERIOD'] = '2022-01-01'

In [34]:
df_churn_sel['PERIOD'] = pd.to_datetime(df_churn_sel['PERIOD'])

In [35]:
df_churn_sel = df_churn_sel[['PERIOD','TRO_MEMBERS']].rename(columns={'TRO_MEMBERS':'MEMBER'})

In [36]:
df_churn_sel.head()

,PERIOD,MEMBER
21,2022-01-01,9990010000089999
26,2022-01-01,9990010000114500
104,2022-01-01,9990010000324539
116,2022-01-01,9990010000345690
128,2022-01-01,9990010000384497


In [ ]:
df_churn_sel_3_3 = df_churn_sel.copy()

In [ ]:
df_churn_sel_3_3.tail()

In [ ]:
df_churn_all = pd.concat([df_churn_sel, df_churn_sel_3_3])

In [ ]:
df_churn_all = df_churn_all.drop_duplicates(subset='MEMBER', keep='last')

In [44]:

df_churn_all = df_churn_sel.iloc[-5000:,:]

In [46]:
df_churn_all.shape

(5000, 2)

In [47]:


from sqlalchemy import event,create_engine,types



driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'mkt' 
password = 'Mkt234'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )
    
engine = create_engine(engine_stmt)

df = df_churn_all.copy()
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('ALFAGIFT_CHURN', engine, index=False, if_exists="append", dtype=dtyp)

engine.dispose()

In [48]:
df_churn_view = pd.merge(df_churn_sel,df_tto_re_pv,left_on='MEMBER',right_on='TRO_MEMBERS', how='left')

In [51]:
df_churn_view[df_churn_view['label'] == 1]

,PERIOD,MEMBER,TRO_MEMBERS,TRO_NET 2021-09,TRO_NET 2021-10,TRO_NET 2021-11,TRO_NET 2021-12,TRO_DH_SPESIAL 2021-09,TRO_DH_SPESIAL 2021-10,TRO_DH_SPESIAL 2021-11,TRO_DH_SPESIAL 2021-12,TRO_NO_ORDER 2021-09,TRO_NO_ORDER 2021-10,TRO_NO_ORDER 2021-11,TRO_NO_ORDER 2021-12,SLA 2021-09,SLA 2021-10,SLA 2021-11,SLA 2021-12,SLA_LATE 2021-09,SLA_LATE 2021-10,SLA_LATE 2021-11,SLA_LATE 2021-12,TRO_VOUCHER_USAGE 2021-09,TRO_VOUCHER_USAGE 2021-10,TRO_VOUCHER_USAGE 2021-11,TRO_VOUCHER_USAGE 2021-12,TRO_PAYMENT_METHOD 2021-09,TRO_PAYMENT_METHOD 2021-10,TRO_PAYMENT_METHOD 2021-11,TRO_PAYMENT_METHOD 2021-12,RATING_BELOW 2021-09,RATING_BELOW 2021-10,RATING_BELOW 2021-11,RATING_BELOW 2021-12,RATING_AVG 2021-09,RATING_AVG 2021-10,RATING_AVG 2021-11,RATING_AVG 2021-12,PV 2021-09,PV 2021-10,PV 2021-11,PV 2021-12,SINCE_REGIS,label
2193,2022-01-01,9990012524655300,9990012524655300,11437721.0,11406749.0,14691750.0,0.0,203900.0,335752.0,489000.0,0.0,168.0,172.0,161.0,0.0,3149.1,2325.1,4611.1,0.0,1.0,1.0,10.0,0.0,0.0,0.0,1156600.0,0.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177.0,143.0,148.0,0.0,797.0,1
2285,2022-01-01,9990012608837600,9990012608837600,3651684.0,3311357.0,6877382.0,0.0,226300.0,274202.0,311702.0,0.0,55.0,50.0,122.0,0.0,970.4,562.8,1301.6,0.0,3.0,1.0,0.0,0.0,50000.0,0.0,55000.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,110.0,91.0,24.0,747.0,1
4341,2022-01-01,9990013255019719,9990013255019719,1308286.0,6408100.0,9411739.0,0.0,50400.0,561500.0,591402.0,0.0,15.0,55.0,86.0,0.0,140.9,1208.9,2853.9,0.0,0.0,4.0,16.0,0.0,0.0,0.0,2695000.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,225.0,378.0,606.0,10.0,303.0,1
6336,2022-01-01,9990013340534198,9990013340534198,5831502.0,333077.0,7968355.0,0.0,403600.0,29300.0,546900.0,0.0,53.0,3.0,54.0,0.0,370.3,27.3,525.6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1895000.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,477.0,742.0,853.0,1.0,188.0,1
7998,2022-01-01,9990013388167259,9990013388167259,54000.0,0.0,52600.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.8,0.0,20.5,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,119.0,1
8877,2022-01-01,9990013416719330,9990013416719330,2370079.0,8690130.0,4328018.0,0.0,140500.0,321500.0,152900.0,0.0,20.0,71.0,24.0,0.0,244.9,730.5,321.4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,920000.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,342.0,324.0,0.0,86.0,1
10155,2022-01-01,9990013439790471,9990013439790471,0.0,46817.0,41180.0,0.0,0.0,200.0,200.0,0.0,0.0,1.0,1.0,1.0,0.0,1256.2,93.9,20.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,54.0,1
